## pg 脚本

+ 补全 历史数据ID字段
+ ID字段创建为主键
+ dev pament db=1

In [1]:
from __future__ import absolute_import, unicode_literals

import json

import psycopg2
import redis
from logging import exception
from pprint import pprint
from collections import defaultdict
import conf

class Connection():
    abstract = True
    _rdb1 = None
    _rdb2 = None
    _pgconn = None

    @property
    def rdb_1(self):
        if self._rdb1 is None:
            self._rdb1 = redis.StrictRedis(host=conf.REDIS_PAYMENT_HOST,
                                           port=conf.REDIS_ACTIVITY_PORT,
                                           db=1,
                                           charset="utf-8", 
                                           decode_responses=True)
        return self._rdb1

    @property
    def rdb_2(self):
        if self._rdb2 is None:
            self._rdb2 = redis.StrictRedis(host=conf.REDIS_PAYMENT_HOST,
                                           port=conf.REDIS_ACTIVITY_PORT,
                                           db=2,
                                           charset="utf-8", 
                                           decode_responses=True)
        return self._rdb2
    
    @property
    def pgconn(self):
        if self._pgconn is None:
            self._pgconn = psycopg2.connect(conf.PG_BOLUOME)
        return self._pgconn
    
    def test(self):
        print('e')
        return self.rdb_1.keys('p:*')
    
c=Connection()

In [8]:
c.rdb_1.exists("p:boluome:1000000082552")

True

In [6]:
c.rdb_1.rename("p:boluome:1000000082552-test","p:boluome:1000000082552")

ResponseError: no such key

In [19]:
def paid_id_fetch(order_id,app_code,rdb):
    return rdb.hgetall('p:{}:{}'.format(app_code,order_id))
def pgpay_add_id():
    conn = psycopg2.connect(conf.PG_BOLUOME)
    with conn:
        with conn.cursor() as cur:
            cur.execute(('SELECT '
                         '  * '
                         'FROM '
                         '  pay '))
            data=cur.fetchall()
            for index,row in enumerate(data):
                order_id=row[0]
                app_code=row[-2]
#                 print(app_code,order_id)
                resp=paid_id_fetch(order_id,app_code,c.rdb_1)
                paid_id=resp.get('id')
                if not resp:
                    return ['err',order_id,app_code]
                else:
                    if not paid_id:
                        paid_id=order_id
                    cur.execute(
                        """UPDATE pay
                           SET id=%s
                        WHERE order_id=%s;""",
                        (paid_id,order_id)
                    )
    return 
# pgpay_add_id()

## REDIS 数据同步
+ 支付 db=1

In [248]:
def redis_pay_task():
    resp=True
    while resp:
        resp=c.rdb_1.lpop('pay_tasks')
        if resp:
            print('redis_pay_task',resp)
            redis_up(resp)
            
#             resp=False
            
    return 'finish'
    
def redis_up(app_code_id,key=None,keys=None):
    if keys is None:
        keys=[]
    if key is None:
        key=True
    while key:
        key=c.rdb_1.spop('up:{}'.format(app_code_id))
        if key:
            keys.append(key)
    if not keys:
        print('up: no data')
        return 'up: no data'
    print('redis_up',app_code_id,keys)
    return redis_p_id(app_code_id,keys)


def redis_p_id(app_code_id,keys):
    values=c.rdb_1.hmget('p:{}'.format(app_code_id),keys)
    data={k:values[i] for i,k in enumerate(keys) if values[i]}
    if not data:
        print('p: no data')
        return 'p: no data'
    data['paidId']=app_code_id.split(':')[-1]
    print(data)
    return pg_upsert(data)

def redis_err_id(paid_id):
    c.rdb_1.rpush('pay_err',paid_id)
    return 'pay_err_id insert ok'

def pg_upsert(data):
    map_key={
        'appCode':'app_code',
        'chargeId':'charge_id',
        'orderType':'order_type',
        'paidAt':'paid_at',
        'payPrice':'pay_price',
        'serialNum':'serial_num',
        'userId':'user_id',
        'paidId':'id',
        'createdAt':'created_at',
        'id':'order_id',
    }
    pg_data={map_key.get(k,k):v for k,v in data.items()}
    conn = psycopg2.connect(conf.PG_BOLUOME)
    try:
        with conn:
            with conn.cursor() as cur:
                if 'order_id' in pg_data:
                    cur.execute("""INSERT INTO pay(
                        order_id, price, pay_price, channel, user_id, charge_id, created_at, 
                        order_type, status, serial_num, app_code, id)
                        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
                        [pg_data.get('order_id'),pg_data.get('price'),pg_data.get('pay_price'),
                        pg_data.get('channel'),pg_data.get('user_id'),pg_data.get('charge_id'),
                        pg_data.get('created_at'),pg_data.get('order_type'),pg_data.get('status'),
                        pg_data.get('serial_num'),pg_data.get('app_code'),pg_data.get('id')]
                    )
                else:
                    sql_set=''
                    for key,value in pg_data.items():
                        sql_set+=key+'='+'%s'+','
                    sql_set=sql_set.strip(',')
                    sql_values=[v for v in pg_data.values()]
                    sql_values.append(pg_data['id'])
                    sql="""UPDATE pay SET {} WHERE id=%s;""".format(sql_set)
                    cur.execute(sql,sql_values)
            conn.commit()
    except Exception as e :
        exception(e)
        redis_err_id(pg_data['id'])
        return 'upsert err ,data error'
    conn.close()
    print(pg_data)
    return 'upsert ok'

In [770]:
# redis_pay_task()

+ 退款 db=2

In [369]:
def redis_pay_task():
    resp=True
    while resp:
        resp=c.rdb_2.lpop('refund_tasks')
        if resp:
            print('redis_pay_task',resp)
            redis_up(resp)
            
#             resp=False
            
    return 'finish'
    
def redis_up(app_code_id,key=None,keys=None):
    if keys is None:
        keys=[]
    if key is None:
        key=True
    while key:
        key=c.rdb_2.spop('up:{}'.format(app_code_id))
        if key:
            keys.append(key)
    if not keys:
        print('up: no data')
        return 'up: no data'
    print('redis_up',app_code_id,keys)
    return redis_r_id(app_code_id,keys)


def redis_r_id(app_code_id,keys):
    values=c.rdb_2.hmget('r:{}'.format(app_code_id),keys)
    data={k:values[i] for i,k in enumerate(keys) if values[i]}
    if not data:
        print('r: no data')
        return 'r: no data'
    data['paidId']=app_code_id.split(':')[-1]
    print(data)
    return pg_upsert(data)

def redis_err_id(paid_id):
    c.rdb_2.rpush('refund_err',paid_id)
    return 'pay_err_id insert ok'

def pg_upsert(data):
    map_key={
        'appCode':'app_code',
        'refundedtimeline':'refunded_timeline',
        'ordertype':'order_type',
        'paidprice':'pay_price',
        'refundedprice':'refunded_price',
        'userid':'user_id',
        'createdat':'created_at',
        'id':'order_id',
        'paidId':'id'
    }
    pg_data={map_key.get(k,k):v for k,v in data.items()}
    pg_data.setdefault('app_code','boluome')
    conn = psycopg2.connect(conf.PG_BOLUOME)
    try:
        with conn:
            with conn.cursor() as cur:
                if 'order_id' in pg_data:
                    cur.execute("""INSERT INTO refund(
                        price, paid_price, refunded_price, timeline, refunded_timeline, 
                        created_at, user_id, order_type, order_id, app_code, id)
                        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
                        [pg_data.get('price'),pg_data.get('paid_price'),pg_data.get('refunded_price'),
                        pg_data.get('timeline'),pg_data.get('refunded_timeline'),pg_data.get('created_at'),
                        pg_data.get('user_id'),pg_data.get('order_type'),pg_data.get('order_id'),
                        pg_data.get('app_code'),pg_data.get('id')]
                    )
                else:
                    sql_set=''
                    for key,value in pg_data.items():
                        sql_set+=key+'='+'%s'+','
                    sql_set=sql_set.strip(',')
                    sql_values=[v for v in pg_data.values()]
                    sql_values.append(pg_data['id'])
                    sql="""UPDATE refund SET {} WHERE id=%s;""".format(sql_set)
                    cur.execute(sql,sql_values)
            conn.commit()
    except Exception as e :
        exception(e)
        redis_err_id(pg_data['id'])
        return 'upsert err ,data error'
    conn.close()
    print(pg_data)
    return 'upsert ok'

In [771]:
# redis_pay_task()

In [384]:
a=[]
b=1
c=a.append(b) if b else 0
a,c
keys=[1,2]
keys = [] if keys is None else keys
keys

[1, 2]

In [387]:
def test():
    return 1 if 0 else 0
test()

0

In [399]:
task_name=0
task = 1 if task_name in ['pay_tasks', 1] else 0
task

0

## 整合版

In [930]:
from __future__ import absolute_import, unicode_literals

import json

import psycopg2
import redis
from logging import exception
from pprint import pprint
import conf


class PgTask():
    abstract = True
    _rdb1 = None
    _rdb2 = None
    _pgconn = None

    @property
    def rdb_1(self):
        if self._rdb1 is None:
            self._rdb1 = redis.StrictRedis(host=conf.REDIS_PAYMENT_HOST,
                                           port=conf.REDIS_PORT,
                                           db=1,
                                           charset="utf-8",
                                           decode_responses=True)
        return self._rdb1

    @property
    def rdb_2(self):
        if self._rdb2 is None:
            self._rdb2 = redis.StrictRedis(host=conf.REDIS_PAYMENT_HOST,
                                           port=conf.REDIS_PORT,
                                           db=2,
                                           charset="utf-8",
                                           decode_responses=True)
        return self._rdb2

    @property
    def pgconn(self):
        if self._pgconn is None:
            self._pgconn = psycopg2.connect(conf.PG_BOLUOME)
        return self._pgconn
p=PgTask()

In [951]:
data=p.rdb_1.hgetall('p:{0}:{1}'.format('boluome', '100000396616')) or p.rdb_1.hgetall('p:{0}:{1}'.format('boluome', '100000396616'+'00'))
pprint(data)
data.get('id','') or data.get('serialNum','')

{'appCode': 'boluome',
 'createdAt': '1488542310002',
 'id': '100000396616',
 'name': '钱包充值 - 0.01元',
 'orderType': 'balance',
 'paidAt': '1488542310002',
 'price': '1',
 'serialNum': '10000039661600',
 'status': '1',
 'type': '0',
 'userId': 'blm_rlco03m'}


'100000396616'

In [773]:
def redis_pay_task(task_name):
    resp = True
    task = 1 if task_name in ['pay_tasks', 1] else 2
    while resp:
        resp = p.rdb_1.lpop('pay_tasks') if task == 1 else p.rdb_2.lpop('refund_tasks')
        redis_up(resp, task) if resp else 0
        resp=False
    return ['task name', task, 'finish']

In [774]:
def redis_up(app_code_id, task_name, key=None, keys=None):
    keys = [] if keys is None else keys
    key = True if key is None else key
    while key:
        if task_name == 1:
            key = p.rdb_1.spop('up:{}'.format(app_code_id))
        elif task_name == 2:
            key = p.rdb_2.spop('up:{}'.format(app_code_id))
        keys.append(key) if key else 0
    return redis_p_id(app_code_id, keys, task_name) if keys else [task_name, 'up: no data']

In [775]:
def redis_p_id(app_code_id, keys, task_name):
    if ':' not in app_code_id:
        app_code_id='boluome:'+app_code_id
    if task_name == 1:
        values = p.rdb_1.hmget('p:{}'.format(app_code_id), keys)
    if task_name == 2:
        values = p.rdb_2.hmget('r:{}'.format(app_code_id), keys)
    data = {k: values[i] for i, k in enumerate(keys) if values[i]}
    if not data:
        return 'p: no data'
    data['paidId'] = app_code_id.split(':')[-1]
    return pg_upsert(data, task_name)

In [820]:
def redis_err_id(paid_id, task_name):
    if task_name == 1:
        p.rdb_1.lrem('pay_err_1', 0, paid_id)
        p.rdb_1.rpush('pay_err_1', paid_id)
        print('pay_err_id insert ok')
        return 'pay_err_id insert ok'
    else:
        p.rdb_2.lrem('paid_err_2', 0, paid_id)
        p.rdb_2.rpush('paid_err_2', paid_id)
        print('refund_err_id insert ok')
        return 'refund_err_id insert ok'

In [850]:
def pg_upsert(data, task_name):
    map_key = {
            'appCode': 'app_code',
            'refundedTimeline': 'refunded_timeline',
            'orderType': 'order_type',
            'paidPrice': 'paid_price',
            'refundedPrice': 'refunded_price',
            'userId': 'user_id',
            'createdAt': 'created_at',
            'id': 'order_id',
            'paidId': 'id',
            'timeline':'timeline',
            'paidAt':'paid_at',
            'chargeId': 'charge_id',
            'paidAt': 'paid_at',
            'payPrice': 'pay_price',
            'serialNum': 'serial_num',
        }
    pg_data = {map_key.get(k, k): v for k, v in data.items()}
    print(pg_data)
    if task_name == 1:
        try:
            with p.pgconn.cursor() as cur:
                if 'order_id' in pg_data:
                    pg_data.setdefault('app_code', 'boluome')
                    cur.execute("""INSERT INTO pay(
                        order_id, price, pay_price, channel, user_id, charge_id, created_at, 
                        order_type, status, serial_num, app_code, id)
                        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
                                [pg_data.get('order_id'), pg_data.get('price'), pg_data.get('pay_price'),
                                    pg_data.get('channel'), pg_data.get('user_id'), pg_data.get('charge_id'),
                                    pg_data.get('created_at'), pg_data.get('order_type'), pg_data.get('status'),
                                    pg_data.get('serial_num'), pg_data.get('app_code'), pg_data.get('id')]
                                )
                else:
                    sql_set = ''
                    sql_values=[]
                    for key, value in pg_data.items():
                        if key in['order_id', 'price', 'pay_price', 'channel', 'user_id', 'charge_id', 'created_at', 
                                'order_type', 'status', 'serial_num', 'app_code', 'id']:
                            sql_set += key + '=' + '%s' + ','
                            sql_values.append(value)
                    sql_set = sql_set.strip(',')
#                     sql_values = [v for v in pg_data.values()]
                    sql_values.append(pg_data['id'])
                    sql = """UPDATE pay SET {} WHERE id=%s;""".format(sql_set)
                    print(sql,sql_values)
                    cur.execute(sql, sql_values)
            p.pgconn.commit()
        except Exception as e:
            p.pgconn.rollback()
            with p.pgconn.cursor() as cur:
                cur.execute("""SELECT * FROM pay WHERE id=%s""",
                            (pg_data.get('id'), ))
                data = cur.fetchone()
            if data:
                pg_data.pop('order_id') if 'order_id' in pg_data else 0
                pg_data['paidId']=pg_data.pop('id')
                pg_upsert(pg_data, task_name)

            else:
                redis_err_id(pg_data['id'], task_name)
                exception(e)
#             exception(e)
#             p.pgconn.rollback()
#             redis_err_id(pg_data['id'], task_name)
            return 'upsert err ,data error'
    if task_name == 2:
        try:
            with p.pgconn.cursor() as cur:
                if 'order_id' in pg_data:
                    pg_data.setdefault('app_code', 'boluome')
                    print(pg_data)
                    cur.execute("""INSERT INTO refund(
                            price, paid_price, refunded_price, timeline, refunded_timeline, 
                            created_at, user_id, order_type, order_id, app_code, id,paid_at)
                            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
                                [pg_data.get('price'), pg_data.get('paid_price'), pg_data.get('refunded_price'),
                                 pg_data.get('timeline'), pg_data.get(
                                     'refunded_timeline'), pg_data.get('created_at'),
                                 pg_data.get('user_id'), pg_data.get('order_type'), pg_data.get('order_id'),
                                 pg_data.get('app_code'), pg_data.get('id'),pg_data.get('paid_at')]
                                )
                else:
                    sql_set = ''
                    sql_values=[]
                    for key, value in pg_data.items():
                        if key in ['price', 'paid_price', 'refunded_price', 'timeline', 'refunded_timeline', 
                                    'created_at', 'user_id', 'order_type', 'order_id', 'app_code', 'id', 'paid_at']:
                            sql_set += key + '=' + '%s' + ','
                            sql_values.append(value)
                    sql_set = sql_set.strip(',')
#                     sql_values = [v for v in pg_data.values()]
                    sql_values.append(pg_data['id'])
                    sql = """UPDATE refund SET {} WHERE id=%s;""".format(sql_set)
                    print(sql,sql_values)
                    cur.execute(sql, sql_values)
            p.pgconn.commit()
        except Exception as e:
            p.pgconn.rollback()
            with p.pgconn.cursor() as cur:
                cur.execute("""SELECT * FROM pay WHERE id=%s""",
                            (pg_data.get('id'), ))
                data = cur.fetchone()
            if data:
                pg_data.pop('order_id') if 'order_id' in pg_data else 0
                pg_data['paidId']=pg_data.pop('id')
                pg_upsert(pg_data, task_name)

            else:
                redis_err_id(pg_data['id'], task_name)
                exception(e)
                
#             exception(e)
#             p.pgconn.rollback()
#             redis_err_id(pg_data['id'], task_name)
            return 'upsert err ,data error'
#         finally:
#             p.pgconn.commit()
    print(task_name, 'upsert ok')
    return [task_name, 'upsert ok']

In [925]:
def err_task_retry(task_name):
    resp = True
    task = 1 if task_name in ['pay_err', 1] else 2
    while resp:
        if task == 1:
            resp = p.rdb_1.lpop('pay_err')
            if not resp:
                return
            resp_id = p.rdb_1.keys('p:*{}'.format(resp))[0]
            if 'records' in resp_id:
                continue
            resp_data = p.rdb_1.hgetall(resp_id)
        if task == 2:
            resp = p.rdb_2.lpop('paid_err')
            if not resp:
                return
            resp_id = p.rdb_2.keys('r:*{}'.format(resp))[0]
            if 'records' in resp_id:
                continue
            resp_data = p.rdb_2.hgetall(resp_id)
        resp_data['paidId'] = resp_id.split(':')[-1]
        print(resp_data)
        pg_upsert(resp_data, task)
#         resp=False
    return 'err_id ok'

In [926]:
err_task_retry(1)

ERROR:root:null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000382649, 100, null, null, blm_jpbkbfm, null, 1466131657495, liuliang, 1, null, boluome, 100000382649        ).
Traceback (most recent call last):
  File "<ipython-input-850-83a2a783e0f3>", line 33, in pg_upsert
    pg_data.get('serial_num'), pg_data.get('app_code'), pg_data.get('id')]
psycopg2.IntegrityError: null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000382649, 100, null, null, blm_jpbkbfm, null, 1466131657495, liuliang, 1, null, boluome, 100000382649        ).

ERROR:root:null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000395054, 13930, null, null, blm_3s17zhc, null, 1486456490924, shenghuojiaofei, 1, null, boluome, 100000395054        ).
Traceback (most recent call last):
  File "<ipython-input-850-83a2a783e0f3>", line 33, in pg_upsert
    pg_data.get('serial_num'), pg_data.get('app

p:boluome:100000382649
{'id': '100000382649', 'createdAt': '1466131657495', 'price': '100', 'status': '1', 'paidAt': '1466131657495', 'orderType': 'liuliang', 'userId': 'blm_jpbkbfm', 'paidId': '100000382649'}
{'order_id': '100000382649', 'created_at': '1466131657495', 'price': '100', 'status': '1', 'paid_at': '1466131657495', 'order_type': 'liuliang', 'user_id': 'blm_jpbkbfm', 'id': '100000382649'}
pay_err_id insert ok
p:boluome:100000395054
{'id': '100000395054', 'appCode': 'boluome', 'createdAt': '1486456490924', 'price': '13930', 'name': '1000000210-0210485654', 'status': '1', 'paidAt': '1486456490924', 'orderType': 'shenghuojiaofei', 'userId': 'blm_3s17zhc', 'paidId': '100000395054'}
{'order_id': '100000395054', 'app_code': 'boluome', 'created_at': '1486456490924', 'price': '13930', 'name': '1000000210-0210485654', 'status': '1', 'paid_at': '1486456490924', 'order_type': 'shenghuojiaofei', 'user_id': 'blm_3s17zhc', 'id': '100000395054'}
pay_err_id insert ok
p:boluome:100000397403


ERROR:root:null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000389019, 4980, null, null, blm001, null, 1477560237812, huafei, 1, null, boluome, 100000389019        ).
Traceback (most recent call last):
  File "<ipython-input-850-83a2a783e0f3>", line 33, in pg_upsert
    pg_data.get('serial_num'), pg_data.get('app_code'), pg_data.get('id')]
psycopg2.IntegrityError: null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000389019, 4980, null, null, blm001, null, 1477560237812, huafei, 1, null, boluome, 100000389019        ).

ERROR:root:null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000389338, 7800, null, null, blm_jnk1d86, null, 1478605503950, jiadianqingxi, 1, null, boluome, 100000389338        ).
Traceback (most recent call last):
  File "<ipython-input-850-83a2a783e0f3>", line 33, in pg_upsert
    pg_data.get('serial_num'), pg_data.get('app_code'), pg_dat

pay_err_id insert ok
p:boluome:100000391841
{'id': '100000391841', 'createdAt': '1482283477012', 'price': '3090', 'status': '1', 'paidAt': '1482283477012', 'orderType': 'dianying', 'userId': '71', 'paidId': '100000391841'}
{'order_id': '100000391841', 'created_at': '1482283477012', 'price': '3090', 'status': '1', 'paid_at': '1482283477012', 'order_type': 'dianying', 'user_id': '71', 'id': '100000391841'}
{'created_at': '1482283477012', 'price': '3090', 'status': '1', 'paid_at': '1482283477012', 'order_type': 'dianying', 'user_id': '71', 'app_code': 'boluome', 'id': '100000391841'}
UPDATE pay SET created_at=%s,price=%s,status=%s,order_type=%s,user_id=%s,app_code=%s,id=%s WHERE id=%s; ['1482283477012', '3090', '1', 'dianying', '71', 'boluome', '100000391841', '100000391841']
1 upsert ok
p:boluome:100000389338
{'id': '100000389338', 'createdAt': '1478605503950', 'price': '7800', 'status': '1', 'paidAt': '1478605503950', 'orderType': 'jiadianqingxi', 'userId': 'blm_jnk1d86', 'paidId': '100

ERROR:root:null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000393784, 4550, null, null, 1845, null, 1484279118987, dianying, 1, null, 91ala, 100000393784        ).
Traceback (most recent call last):
  File "<ipython-input-850-83a2a783e0f3>", line 33, in pg_upsert
    pg_data.get('serial_num'), pg_data.get('app_code'), pg_data.get('id')]
psycopg2.IntegrityError: null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000393784, 4550, null, null, 1845, null, 1484279118987, dianying, 1, null, 91ala, 100000393784        ).

ERROR:root:null value in column "channel" violates not-null constraint
DETAIL:  Failing row contains (100000380388, 3590, null, null, blm_3rego9v, null, 1463629289758, dianying, 1, null, boluome, 100000380388        ).
Traceback (most recent call last):
  File "<ipython-input-850-83a2a783e0f3>", line 33, in pg_upsert
    pg_data.get('serial_num'), pg_data.get('app_code'), pg_data.get('id

{'id': '100000393784', 'appCode': '91ala', 'createdAt': '1484279118987', 'price': '4550', 'name': '你的名字。', 'status': '1', 'paidAt': '1484279118987', 'orderType': 'dianying', 'userId': '1845', 'paidId': '100000393784'}
{'order_id': '100000393784', 'app_code': '91ala', 'created_at': '1484279118987', 'price': '4550', 'name': '你的名字。', 'status': '1', 'paid_at': '1484279118987', 'order_type': 'dianying', 'user_id': '1845', 'id': '100000393784'}
pay_err_id insert ok
p:boluome:100000380388
{'id': '100000380388', 'createdAt': '1463629289758', 'price': '3590', 'status': '1', 'paidAt': '1463629289758', 'orderType': 'dianying', 'userId': 'blm_3rego9v', 'paidId': '100000380388'}
{'order_id': '100000380388', 'created_at': '1463629289758', 'price': '3590', 'status': '1', 'paid_at': '1463629289758', 'order_type': 'dianying', 'user_id': 'blm_3rego9v', 'id': '100000380388'}
pay_err_id insert ok
p:boluome:100000390970
{'id': '100000390970', 'createdAt': '1480169756586', 'price': '7000', 'status': '1', 'p

ResponseError: WRONGTYPE Operation against a key holding the wrong kind of value

In [695]:
redis_pay_task(1)

{'serial': '1000003886701476695061368', 'created_at': '1476695061199', 'price': '1', 'status': '1', 'user_id': 'blm_35xw3cc', 'order_type': 'balance', 'channel': 'ccb', 'order_id': '100000388670', 'id': '100000388670'}
1 upsert ok


['task name', 1, 'finish']

In [849]:
cur=p.pgconn.cursor()
cur.execute("UPDATE pay SET pay_price=%s,status=%s,app_code=%s,price=%s,user_id=%s,created_at=%s,channel=%s,order_type=%s,charge_id=%s,id=%s WHERE id=%s;", ['20', '中石油卡:903****3452 充值 0.20元', '2', '1489543668221', 'boluome', '20', 'test_long', '1489543662911', 'alipay_wap', 'jiayouka', 'ch_bTOyPCGebvH4bX5yfTLOGG08', '1000003971341489543666133', '{"success_url":"http://boluome.test.otosaas.com/pay/success/100000397134"}', '100000397134', '100000397134'])

TypeError: not all arguments converted during string formatting

In [45]:
class test():
    _u=2
    def __init__(self):
        self._num=None
        
    def num(self):
        if self._num is None:
            self._num=0
        else:
            self._num+=1
        return self._num
t=test()

In [48]:
t.num()

2

In [56]:
tt=test()
tt.num()

0

In [20]:
t._u+=1
t._u


4

In [65]:
import time
time.mktime(time.strptime('2017-04-01',"%Y-%m-%d"))
int(time.time())

1492085561

In [68]:
keys=[1,2,3,5,8,10]
[num for num in range(32) if num not in keys]

[0,
 4,
 6,
 7,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [103]:
time_stamp = int(time.time()) - 1490976000
print(time_stamp,time_stamp << 1)
(1<<4)
(time_stamp << 17)+(5<<12)+(40)
# print(int(str(time_stamp),2))
int(20,2)

1165305 2330610


TypeError: int() can't convert non-string with explicit base

In [125]:
class Application():
    abstract = True

    def __init__(self):
        self._rdb = None
        self._key = None
        self._number = 0


    @property
    def rdb(self):
        if self._rdb is None:
            self._rdb = redis.StrictRedis(host=conf.STG_MONGO_HOST,
                                          port=conf.REDIS_PORT,
                                          db=1,
                                          charset="UTF-8",
                                          decode_responses=True)
        return self._rdb

#     @property
#     def key(self):
#         """机器号"""
#         if self._key is None:
#             keys = self.rdb.hkeys('server_bind')
#             # keys = self.rdb.hset('server_bind', 'test', 100)
#             return keys
#             for num in range(32):
#                 if num not in keys:
#                     self._key = num
#                     break
#             if self._key is None:
#                 for key in keys:
#                     value = self.rdb.hget('server_bind', key)
#                     data = requests.get(value)
#                     if data.get('code') in [200, '200']:
#                         continue
#                     else:
#                         self._key = key
#                         break
#             if self._key is None:
#                 return

#             self.rdb.hset('server_bind', key, 'http://127.0.0.1/app/{}/ping'.format(self._key))
#             # handlers.append((r'/app/{}/ping'.format(self._key), PHandler))
#         return self._key
a=Application()